# XU Method – Entropy-Based Anomaly Detection on CTU-13

Ce Notebook démontre l'approche XU, basée sur l'entropie, sur un dataset de type CTU-13.

## Fonctions utilisés

### 1. Import & Setup

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tqdm.notebook import tqdm
import json
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline
sns.set()
print("Libraries loaded.")

#Fonction pour afficher les données
from IPython.display import display, HTML

def display_scrollable_dataframe(df, max_height=400):
    display(HTML(df.to_html(notebook=True)))
    display(HTML(f"""
    <style>
    table {{
        display: block;
        max-height: {max_height}px;
        overflow-y: scroll;
        border: 1px solid #ccc;
    }}
    </style>
    """))



Libraries loaded.


### Chargements des données et Preprocessing

In [ ]:
def preprocess_binetflows(df):
    """
    Harmonise le format du dataset aux standards des algorithmes XU et de classification utilisés précédemment,
    en renommant uniquement les colonnes nécessaires, sans toucher aux colonnes :
    'State', 'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes'.

    Conserve le label d'origine dans une colonne 'Label_description'.

    Paramètres :
    - df : DataFrame brut

    Retour :
    - df_clean : DataFrame nettoyé et harmonisé
    """

    print("🔧 Renommage des colonnes au format standard (sans toucher à 'State', 'sTos', 'dTos', etc)...")
    rename_map = {
        'StartTime': 'flow_start',
        'Dur': 'Durat',
        'Proto': 'Prot',
        'SrcAddr': 'SrcIP',
        'Sport': 'SrcPort',
        'Dport': 'DstPort',
        'Dir': 'Direc',  # si tu veux garder l'info de direction
        'DstAddr': 'DstIP'
        # Pas de renommage pour : State, sTos, dTos, TotPkts, TotBytes, SrcBytes
    }

    df_clean = df.rename(columns=rename_map)

    # Conserver les labels d'origine dans une nouvelle colonne
    print("📋 Sauvegarde du label d'origine dans 'Label_description'...")
    df_clean['Label_description'] = df_clean['Label'].astype(str)

    # Harmonisation des labels simplifiés
    print("🧼 Nettoyage des labels (uniquement Background, Normal, Botnet)...")
    df_clean['Label'] = df_clean['Label_description'].apply(lambda x: (
        'Botnet' if 'Botnet' in x else
        'Normal' if 'Normal' in x else
        'Background'
    ))

    # Conversion de flow_start en datetime
    if not pd.api.types.is_datetime64_any_dtype(df_clean['flow_start']):
        print("📅 Conversion de 'flow_start' en datetime...")
        df_clean['flow_start'] = pd.to_datetime(df_clean['flow_start'], errors='coerce')

    print("✅ Dataset prêt pour les traitements XU et classification.")
    return df_clean


In [ ]:
def hist_of_label_values(df):
    """
    Affiche un histogramme de la répartition des labels dans le DataFrame,
    avec les pourcentages affichés sur les barres.
    
    Paramètres :
    - df : pd.DataFrame contenant une colonne 'Label' avec les valeurs 
           'Background', 'Normal' et 'Botnet'.
    
    Affichage :
    - Un histogramme avec les pourcentages sur les barres.
    - Un dictionnaire affiché contenant les pourcentages.
    """
    # Regroupement des valeurs
    label_values = df['Label'].value_counts()

    # Somme des catégories
    background_count = sum(label_values[label] for label in label_values.index if "Background" in label)
    normal_count = sum(label_values[label] for label in label_values.index if "Normal" in label)
    botnet_count = sum(label_values[label] for label in label_values.index if "Botnet" in label)
    
    # Dictionnaire des comptages
    label_repartition = {
        "Background traffic": background_count, 
        "Normal traffic": normal_count, 
        "Botnet traffic": botnet_count
    }

    # Calcul du total et des pourcentages
    total_traffic = sum(label_repartition.values())
    percentage_of_traffic = {k: round((v / total_traffic) * 100, 2) for k, v in label_repartition.items()}

    # Affichage de l'histogramme
    fig, ax = plt.subplots(figsize=(7,5))
    bars = ax.bar(label_repartition.keys(), label_repartition.values(), color=['gray', 'blue', 'red'])

    # Ajout des pourcentages au-dessus des barres
    for bar in bars:
        height = bar.get_height()
        percentage = (height / total_traffic) * 100
        ax.text(bar.get_x() + bar.get_width()/2, height + total_traffic * 0.02, 
                f"{percentage:.2f}%", ha='center', fontsize=12, fontweight='bold')

    # Amélioration du visuel
    ax.set_ylabel("Nombre de flux", fontsize=12)
    ax.set_title("Répartition du trafic par catégorie", fontsize=14, fontweight='bold')
    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.show()


In [5]:
def plot_label_distribution_separated(df_classified_netflows, time_col='TimeWindow', x_tick_spacing=5, label_to_show=['Botnet', 'Normal', 'Background']):
    """
    Affiche un graphe séparé (non empilé) du nombre de flux pour chaque label par TimeWindow.

    Paramètres :
    - df_classified_netflows : DataFrame contenant 'TimeWindow' et 'Label'
    - time_col : nom de la colonne temporelle (par défaut 'TimeWindow')
    - x_tick_spacing : espacement des ticks X (ex: 5 = 1 tick toutes les 5 TimeWindows)
    """
    print("📊 ➤ Calcul de la distribution des labels par TimeWindow (graphes séparés)...")

    # Conversion TimeWindow en label lisible
    df_classified_netflows['TimeLabel'] = df_classified_netflows[time_col].dt.strftime('%H:%M')

    # Comptage des labels par TimeWindow
    label_counts = df_classified_netflows.groupby(['TimeLabel', 'Label']).size().unstack(fill_value=0)

    # Ajouter les labels manquants si nécessaire
    for label in label_to_show:
        if label not in label_counts.columns:
            label_counts[label] = 0

    label_counts = label_counts[label_to_show]

    # Création des sous-graphes
    n = len(label_to_show)
    fig, axes = plt.subplots(n, 1, figsize=(18, 4 * n), sharex=True)

    if n == 1:
        axes = [axes]

    # Espacement des ticks X
    x_ticks = np.arange(0, len(label_counts), x_tick_spacing)
    x_labels = label_counts.index[x_ticks]

    for i, label in enumerate(label_to_show):
        axes[i].bar(label_counts.index, label_counts[label], color=('red' if label == 'Botnet' else 'blue' if label == 'Normal' else 'gray'))
        axes[i].set_title(f"Activité '{label}' au fil du temps", fontsize=14)
        axes[i].set_ylabel("Nombre de flux")
        axes[i].grid(True, axis='y', linestyle='--', alpha=0.6)
        axes[i].set_xticks(x_ticks)
        axes[i].set_xticklabels(x_labels, rotation=45, ha='right', fontsize=9)

    axes[-1].set_xlabel("Heure (TimeWindow)", fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
def show_botnet_ip_adress(df):
    print("Adresses IP des Botnets")
    display_scrollable_dataframe(pd.DataFrame(df[df["Label"] == "Botnet"]["SrcIP"].value_counts()).head(20))

### Algorithme XU

#### Algorithme d'extraction des clusters significatif par rapport à l'IP

In [ ]:
def compute_relative_uncertainty(prob_dist):
    probs = np.array(prob_dist)
    if len(probs) <= 1:
        return 0.0
    entropy = -np.sum(probs * np.log2(probs + 1e-12))
    max_entropy = np.log2(len(probs))
    return entropy / max_entropy

def extract_significant_clusters_with_live_plot(df, column='SrcIP', alpha0=0.02, beta=0.9, debug=False, live_plot=True):
    """
    Extraction des clusters significatifs + affichage final des courbes à la fin de l’algorithme.
    """
    print(f"📊 ➤ Extraction des clusters significatifs sur colonne '{column}'")

    freqs = df[column].value_counts(normalize=True)
    A = freqs.index.tolist()
    PA = freqs.to_dict()

    S = set()
    R = set(A)
    k = 0
    alpha = alpha0

    log = []
    total_values_list = []
    significant_values_list = []
    alpha_list = []

    PR = [PA[val] for val in R]
    theta = compute_relative_uncertainty(PR)

    while theta <= beta:
        alpha = alpha0 * (0.5 ** k)
        k += 1

        move_to_S = {val for val in R if PA[val] >= alpha}
        S.update(move_to_S)
        R -= move_to_S

        # ✅ Normalisation de PR
        PR_probs = [PA[val] for val in R]
        PR_probs = np.array(PR_probs)
        PR_probs = PR_probs / PR_probs.sum() if PR_probs.sum() > 0 else np.array([1.0])  # éviter division par 0

        theta = compute_relative_uncertainty(PR_probs)

        if debug:
            print(f"\n🔁 Iteration {k}")
            print(f"  ➤ Alpha: {round(alpha, 5)}")
            print(f"  ➤ RU(R): {round(theta, 5)}")
            print(f"  ➤ |S|={len(S)} | |R|={len(R)} | Move_to_S={len(move_to_S)}")

        total_values_list.append(len(A))
        significant_values_list.append(len(S))
        alpha_list.append(alpha)
        log.append({
            'Iteration': k,
            'Alpha': alpha,
            'RU_remaining': theta,
            'Total_Values': len(A),
            'Significant_Values': len(S),
        })

    alpha_star = alpha

    # Ajout des clusters dans le dataset
    df_clustered = df.copy()
    df_clustered['ClusterType'] = df_clustered[column].apply(lambda x: 'Significatif' if x in S else 'Bruit')
    df_log = pd.DataFrame(log)

    print(f"\n✅ Extraction terminée : {len(S)} clusters significatifs extraits. α* final = {alpha_star}")
    print("📌 Liste des valeurs significatives (S) :")
    for i, val in enumerate(sorted(S)):
        print(f"   - {i+1:03d}: {val}")

    if live_plot:
        # 📈 Plots à la fin seulement
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

        iterations = list(range(1, k + 1))

        ax1.plot(iterations, total_values_list, 'b--', label='Total values')
        ax1.plot(iterations, significant_values_list, 'r-', label='Significant values')
        ax1.set_yscale('log')
        ax1.set_title(" Total vs Significant Values")
        ax1.set_xlabel("Iteration")
        ax1.set_ylabel("Number of values (log scale)")
        ax1.legend()
        ax1.grid(True)

        ax2.plot(iterations, alpha_list, 'r-', label='Alpha threshold')
        ax2.set_title(" Évolution du seuil α")
        ax2.set_xlabel("Iteration")
        ax2.set_ylabel("Alpha")
        ax2.grid(True)

        plt.show()

    return S, R, alpha_star, df_clustered, df_log


#### Fonction XU de calcul de distance par rapport aux BC

In [ ]:
def normalized_entropy(series):
    """
    Calcule l'entropie normalisée (Relative Uncertainty - RU).
    """
    counts = series.value_counts(normalize=True, dropna=False)
    entropy = -(counts * np.log2(counts + 1e-12)).sum()
    distinct = len(counts)
    total = len(series)

    if distinct <= 1 or total <= 1:
        return 0.0

    max_entropy = np.log2(min(distinct, total))
    return float(entropy / max_entropy)

def XU_algorithm(df, ref_profiles):
    """
    Algorithme XU.
    Calcul des distances comme la somme des carrés des différences d'entropies normalisées (RU).

    Paramètres :
    - df : DataFrame NetFlows (doit contenir 'SrcIP', 'SrcPort', 'DstPort', 'DstIP', 'Label')
    - ref_profiles : dict des profils de référence : {'nom': [RU_srcPort, RU_dstPort, RU_dstIP]}

    Retour :
    - df_features : DataFrame contenant RU, distances aux profils, et score d’anomalie
    """

    results = []
    ip_groups = df.groupby('SrcIP')

    print(f"➤ Traitement de {len(ip_groups)} adresses IP sources...")

    for src_ip, df_src in tqdm(ip_groups, desc="Analyse XU"):
        ru_srcport = normalized_entropy(df_src['SrcPort'].fillna('N/A'))
        ru_dstport = normalized_entropy(df_src['DstPort'].fillna('N/A'))
        ru_dstip   = normalized_entropy(df_src['DstIP'].fillna('N/A'))

        label = 'Botnet' if 'Botnet' in df_src['Label'].unique() else (
                'Normal' if 'Normal' in df_src['Label'].unique() else 'Background'
        )

        ru_vector = np.array([ru_srcport, ru_dstport, ru_dstip])

        distances = {}
        for profile_name, profile_vector in ref_profiles.items():
            dist = ((ru_vector - profile_vector) ** 2).sum()  # sans racine carrée
            distances[f'Distance_{profile_name.capitalize()}'] = dist

        # Anomaly Score = minimum distance aux profils connus
        min_distance = min(distances.values())

        result_row = {
            'SrcIP': src_ip,
            'RU_SrcPort': ru_srcport,
            'RU_DstPort': ru_dstport,
            'RU_DstIP': ru_dstip,
            'AnomalyScore': min_distance,
            'Label': label,
            'ClusterType': df_src['ClusterType'].values[0] if 'ClusterType' in df_src.columns else 'N/A'
        }
        result_row.update(distances)
        results.append(result_row)

    df_features = pd.DataFrame(results)
    print("✅ XU terminé.")
    return df_features



def apply_threshold_and_categorize_ru(df_features, threshold=0.5, epsilon_dict=None):
    """
    - Applique le seuil de classification sur la colonne 'Mean_Distance'
    - Catégorise chaque RU en Low (0), Medium (1), High (2) selon epsilon_dict
    - Insère les colonnes RU_*_Level juste après leur colonne RU_* correspondante

    Paramètres :
    - df_features : DataFrame avec les colonnes RU_* et Mean_Distance
    - threshold : seuil pour prédiction de 'Botnet' ou 'Normal'
    - epsilon_dict : dictionnaire des epsilon, ex: {'RU_SrcPort': 0.2, 'RU_DstPort': 0.2, 'RU_DstIP': 0.3}

    Retour :
    - df : DataFrame enrichi avec 'Predicted' + colonnes de niveau
    """

    df = df_features.copy()
    df['Predicted'] = df['AnomalyScore'].apply(lambda d: 'Botnet' if d < threshold else 'Normal')

    if epsilon_dict is None:
        epsilon_dict = {
            'RU_SrcPort': 0.2,
            'RU_DstPort': 0.2,
            'RU_DstIP': 0.3
        }

    def categorize_ru(ru_value, epsilon):
        if ru_value <= epsilon:
            return 0  # Low
        elif ru_value >= 1 - epsilon:
            return 2  # High
        else:
            return 1  # Medium

    # Insérer chaque RU_Level juste après la RU correspondante
    for col in ['RU_SrcPort', 'RU_DstPort', 'RU_DstIP']:
        if col in df.columns:
            level_col = col + '_Level'
            df[level_col] = df[col].apply(lambda x: categorize_ru(x, epsilon_dict.get(col, 0.2)))
            
            # Réorganiser les colonnes : insérer level_col juste après col
            cols = list(df.columns)
            col_idx = cols.index(col)
            # Retirer puis insérer à la bonne position
            cols.remove(level_col)
            cols.insert(col_idx + 1, level_col)
            df = df[cols]

    return df




#### Fonctions d'appartenance aux BC

In [ ]:
def apply_threshold_and_categorize_ru(df_features, threshold=0.5, epsilon_dict=None):
    """
    - Applique le seuil de classification sur la colonne 'Mean_Distance'
    - Catégorise chaque RU en Low (0), Medium (1), High (2) selon epsilon_dict
    - Insère les colonnes RU_*_Level juste après leur colonne RU_* correspondante

    Paramètres :
    - df_features : DataFrame avec les colonnes RU_* et Mean_Distance
    - threshold : seuil pour prédiction de 'Botnet' ou 'Normal'
    - epsilon_dict : dictionnaire des epsilon, ex: {'RU_SrcPort': 0.2, 'RU_DstPort': 0.2, 'RU_DstIP': 0.3}

    Retour :
    - df : DataFrame enrichi avec 'Predicted' + colonnes de niveau
    """

    df = df_features.copy()
    df['Predicted'] = df['AnomalyScore'].apply(lambda d: 'Botnet' if d < threshold else 'Normal')

    if epsilon_dict is None:
        epsilon_dict = {
            'RU_SrcPort': 0.2,
            'RU_DstPort': 0.2,
            'RU_DstIP': 0.3
        }

    def categorize_ru(ru_value, epsilon):
        if ru_value <= epsilon:
            return 0  # Low
        elif ru_value >= 1 - epsilon:
            return 2  # High
        else:
            return 1  # Medium

    # Insérer chaque RU_Level juste après la RU correspondante
    for col in ['RU_SrcPort', 'RU_DstPort', 'RU_DstIP']:
        if col in df.columns:
            level_col = col + '_Level'
            df[level_col] = df[col].apply(lambda x: categorize_ru(x, epsilon_dict.get(col, 0.2)))
            
            # Réorganiser les colonnes : insérer level_col juste après col
            cols = list(df.columns)
            col_idx = cols.index(col)
            # Retirer puis insérer à la bonne position
            cols.remove(level_col)
            cols.insert(col_idx + 1, level_col)
            df = df[cols]

    return df

### Fonctions d'extraction de clusters et d'affichage des clusters

In [ ]:
def find_best_k_with_silhouette(df_xu, k_range=range(2, 11)):
    """
    Applique KMeans uniquement sur les IP Botnet, avec les features RU, et calcule les scores de silhouette.
    Génère aussi automatiquement les profils des clusters (custom_profiles) au format prêt à copier.

    Paramètres :
    - df_xu : DataFrame contenant les résultats XU (avec 'Label', 'RU_SrcPort', 'RU_DstPort', 'RU_DstIP')
    - k_range : plage de valeurs de k à tester (par défaut 2 à 10)

    Retour :
    - silhouette_scores : dict {k: score}
    - best_k : valeur de k avec meilleur score
    - kmeans_model : modèle KMeans entraîné avec best_k
    - df_botnet_clustered : df des IP Botnet avec leur cluster associé
    - custom_profiles : dictionnaire des centroïdes au format {nom: vecteur RU}
    """
    print("🔍 ➤ Filtrage des IP Botnet pour clustering...")
    df_botnet = df_xu[df_xu['Label'] == 'Botnet'].copy()
    features = df_botnet[['RU_SrcPort', 'RU_DstPort', 'RU_DstIP']].values

    silhouette_scores = {}
    best_score = -1
    best_k = None
    best_model = None

    print("📈 ➤ Évaluation des scores de silhouette...")
    for k in tqdm(k_range):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        cluster_labels = kmeans.fit_predict(features)
        score = silhouette_score(features, cluster_labels)
        silhouette_scores[k] = score
        print(f"  ➤ k={k}, silhouette={round(score, 4)}")
        if score > best_score:
            best_score = score
            best_k = k
            best_model = kmeans

    # Affichage du graphe silhouette
    plt.figure(figsize=(10, 5))
    plt.plot(list(silhouette_scores.keys()), list(silhouette_scores.values()), marker='o', linestyle='-')
    plt.title("📊 Score de silhouette pour chaque nombre de clusters k")
    plt.xlabel("Nombre de clusters (k)")
    plt.ylabel("Score de silhouette")
    plt.grid(True)
    plt.xticks(list(silhouette_scores.keys()))
    plt.tight_layout()
    plt.show()

    # Appliquer le meilleur clustering sur les IP Botnet
    df_botnet['Cluster'] = best_model.predict(features)

    print(f"\n✅ Meilleur nombre de clusters : k={best_k} (score={round(best_score, 4)})")

    # Génération des profils
    print("\n📦 ➤ Custom profiles (copier-coller ready au format JSON-like):")
    custom_profiles = {}
    for i, centroid in enumerate(best_model.cluster_centers_):
        profile_name = f"cluster_{i}"
        profile_vector = np.round(centroid, 3).tolist()
        custom_profiles[profile_name] = profile_vector

    print("custom_profiles = ")
    print(json.dumps(custom_profiles, indent=4))

    return silhouette_scores, best_k, best_model, df_botnet, custom_profiles
